In [1]:
import mysql.connector
from mysql.connector import Error
import os
import logging
from datetime import datetime
from tqdm import tqdm
import logging

In [2]:
# Configure logging similar to the provided data_ingestion example
logging.basicConfig(filename='weather_data_statistics_log.log',  # Log file path
                    filemode='a',  # Append mode
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

In [3]:
# Database configuration details, Please replace the host, user, password, database name accordingly.
db_config = {
    'host': 'localhost',
    'user': '', 
    'password': '',  
    'database': 'corteva'  
}

In [4]:
def db_connection(config):
    """A function to establish MySQL database connection."""
    try:
        connection = mysql.connector.connect(**config)
        return connection
    except mysql.connector.Error as error:
        logging.error(f"Error connecting to the database: {error}")
    return None

In [5]:
def weather_data_statistics(config):
    """
    Function to calculate average temperature (minimum and maximum) and total precipitation from weather_data table
    and ingest it into a new table in the same database, called weather_statistics.
    
    Args:
        config: Configuration object for database connection.
        
    Returns:
        None
    """
    conn = db_connection(config)  # Assuming db_connection is a function that connects to the database and returns a connection object
    if not conn:
        logging.error("Failed to connect to the database.")
        return
    
    start_time = datetime.now()
    logging.info("Starting weather data statistics calculation...")

    try:
        cursor = conn.cursor()
        # Calculate yearly statistics for each station, excluding missing data
        query = """
        SELECT station_id, 
               YEAR(date) as year, 
               AVG(max_temp) as avg_max_temp, 
               AVG(min_temp) as avg_min_temp, 
               SUM(precipitation) as total_precipitation
        FROM weather_data
        WHERE max_temp IS NOT NULL AND 
              min_temp IS NOT NULL AND 
              precipitation IS NOT NULL
        GROUP BY station_id, YEAR(date)
        """
        cursor.execute(query)
        results = cursor.fetchall()

        # Assuming tqdm is used for visual progress indication
        for row in tqdm(results, desc="Updating statistics"):
            station_id, year, avg_max_temp, avg_min_temp, total_precipitation = row
            upsert_query = """
            INSERT INTO weather_statistics (station_id, year, avg_max_temp, avg_min_temp, total_precipitation)
            VALUES (%s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE avg_max_temp=VALUES(avg_max_temp), 
                                    avg_min_temp=VALUES(avg_min_temp),
                                    total_precipitation=VALUES(total_precipitation)
            """
            cursor.execute(upsert_query, (station_id, year, avg_max_temp, avg_min_temp, total_precipitation))
        conn.commit()
    except Error as e:
        logging.error(f"Database error: {e}")
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
            logging.info("Database connection closed.")

    end_time = datetime.now()
    duration = end_time - start_time
    logging.info(f"Statistics calculation completed. Start time: {start_time}, End time: {end_time}, Duration: {duration}")


In [6]:
if __name__ == "__main__":
    weather_data_statistics(db_config)

Updating statistics: 100%|████████████████████████████████████████████████████████████████████| 4791/4791 [00:02<00:00, 1761.41it/s]
